<a href="https://colab.research.google.com/github/makhmudovamunira/DataScience_Mohirdev/blob/main/MLgaTayyorgarlik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3-QADAM. Ma'lumotlarni Machine Learning uchun tayyorlash
Keyingi katta qadam ma'lumotlarni ML uchun tayyorlash. Avvalgi qadamlarda biz ma'lumotlarga to'g'ridan-to'g'ri ishlov berib, ularni tahlil qildik. O'zimiz uchun ba'zi kerakli xulosalarni oldik. Qaysidur ustunlardagi qiymatlarni o'zgartirtish, yangi ustunlar yaratish kabi ishlarni rejalashtirib oldik. Umuman olganda, ma'lumotlarg bo'lgan talabni shakllantridik. Navbat shu talablar asosida ma'lumotlarni tayyorlash.

Buning uchun endi biz ma'lumotlar bilan to'g'ridan-to'g'ri emas, funksiyalar orqali ishlaymiz. Ya'ni, avvalgi qismda bajargan barcha jarayonlarnbi funksiyalar yordamida avtomatlashtiramiz.
Bunday qilishimizdan maqsad esa, kelajakda yangi ma'lumotlarga ishlov berishda tayyor funksiyani ishga tushirish orqali ma'lumotlarni bittada bizga kerak ko'rinishga keltirib olamiz.

In [ ]:
import pandas as pd
import numpy as np
import sklearn

In [ ]:
URL = "https://github.com/ageron/handson-ml2/blob/master/datasets/housing/housing.csv?raw=true"
df = pd.read_csv(URL)
df.head()

Ma'lumotlarni train va test setga ajratamiz.
1. Oddiy (muvozanatsiz) train/test set

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

2. Muvozanatli (Stratified) train/test set

In [ ]:
df['income_cat']=pd.cut(df['median_income'], bins=[0., 1.5, 3., 4.5, 6., np.inf], labels=[1,2,3,4,5])

from sklearn.model_selection import StratifiedShuffleSplit
stratified_split=StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

for train_index, test_index in stratified_split.split(df, df['income_cat']):
  strat_train_set=df.loc[train_index]
  strat_test_set=df.loc[test_index]


strat_train_set.drop('income_cat', axis=1, inplace=True)
strat_test_set.drop('income_cat', axis=1, inplace=True)

3. Ishlash uchun muvozanatli setlarni tanlaymiz.

Bu safar `label` (basorat qilishimiz kerak bo'lgan) ustun `median_house_value` ni ham alohida ajratamiz.
Bunga sabab, qolgan ustunlarga qo'llangan o'zgarishlarni labelga ham qo'llamaslik.

In [ ]:
housing=strat_train_set.drop('median_house_value', axis=1)
housing_labels=strat_train_set['median_house_value'].copy()

![Imgur](https://i.imgur.com/lO0yL15.png)

### 3.1 Tozalash

In [ ]:
df.info()

Esingizda bo'lsa, `df.info()` yordamida `total_bedrooms ` ustunida ba'zi qatorlarda qiymatlar yo'q (`NaN`) ekanini aniqlagan edik.

Ushbu qatorlar bizga halaqit bermasligi uchun oldimizda 3 ta yo'l bor:
1. Qatorlarni (hududlarni) o'chirib tashlash
2. Bu ustunni umuman tashlab yuborish
3. `NaN` qiymatlarni to'ldirish (`0`, o'rtacha yoki median qiymat bilan)

Bulardan qay birini tanlash, loyihadan loyihaga qarab o'zgaradi. Agar birinchi ikki usulni tansalangiz, buning yo'li oson:

In [ ]:
# 1-variant. Qatorlarni tashlab yuborish
housing.dropna(subset='total_bedrooms')# inplace=True qilishni unutmang

In [ ]:
# 2-variant. Ustunni tashlab yuborish
housing.drop('total_bedrooms',axis=1) # inplace=True

In [ ]:
# 3-variant. Median qiymat bilan to'ldirish
median=housing['total_bedrooms'].median()
housing['total_bedrooms'].fillna(median)

Biz 3-variantni tanlaymiz va buni biroz takomillashtiramiz.

Gap shundaki, bu safar bizga kelgan datasetda `total_bedrooms` ustunida `NaN` qiymatlar bor edi, lekin kelgusida yangi ma'lumotlar kelganda, boshqa, deylik `population` ustunida `NaN` qiymatlar bo'lishi mumkin. Biz yuqorida ma'lumotlarga ishlov berishni avtomatlashtirish haqida gapirdik, demak bizning funksiyamiz, har qanday ustundagi `NaN` qiymatlarni to'ldirib ketishi kerak.

Buning uchun `sklearn` tarkibida maxsus `SimpleImputer` funksiyasi mavjud.

In [ ]:
# SimpleImputer klassini chaqiramiz
from sklearn.impute import SimpleImputer

# SimpleImputer dan yangi obyekt yaratamiz va va NaN qiymatlarni median qiymatlarni median qiymatlar bilan to'ldirishini ko'rsatamiz

imputer=SimpleImputer(strategy='median')

Dataset uchun median qiymatni hisoblash uchun `imputer` obyekti ichidagi `.fit()` metodiga murojaat qilamiz.
Dataset ustunlariga e'tibor bersak, oxirgi `ocean_proximity` ustuni sonli emas. `imputer` esa faqatgina sonli qiymatlar bilan ishlaydi (sonsiz qiymatlarnini o'rta qiymati mavjud emas).

In [ ]:
imputer.fit(housing)

Yuqoridagi xatoning oldini olish uchun, sonli ustunlarni alohida ajratib olamiz:

In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)
imputer.fit(housing_num)

In [ ]:
imputer.fit(housing_num)

Hisoblangan median qiymatlar `statistics_` atrributida saqlanadi:

In [ ]:
imputer.statistics_

In [ ]:
X=imputer.transform(housing_num)
X

In [ ]:
type(X)

 `.transform()` funksiyasi numpy array qaytaradi. Arrayni qaytarib dataframega o'tkazamiz:

In [ ]:
housing_tr=pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)
housing_tr.head()

### 3.2 Matnli ustunlarni shakl almashtirish
Datasetimizning `ocean_proximity` ustuni matnli ustun. Keling boshlanishga bu ustunni ajratib olamiz.

In [ ]:
housing_cat=housing[['ocean_proximity']]

In [ ]:
housing_cat['ocean_proximity'].head(10)

 ML algoritmlar sonlar bilan ishlaydi. Shuning uchun bu ustunni ham sonlarga o'zgartirishimiz kerak.

Buning 2 usuli bor:
1. Har bir matnni son bilan almashtirish.

Buning uchun sklearn tarkibida `OrdinalEncoder`dan foydalanamiz:

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder=OrdinalEncoder()
housing_cat_encoder=ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoder[:10]

In [ ]:
ordinal_encoder.categories_

Bizda, ahamiyat bersangiz qo'shni qiymatlar (matnlar) bir-biriga yaqin emas. `<1H OCEAN` va `INLAND` ga `0.0` va `1.0` qiymatlari berilgan, vaholanki, mantiqan o'ylansa bu ikkisi bir-biridan uzoq. `NEAR BAY`, `NEAR OCEAN` qiymatlari esa aksincha, `<1H OCEAN`dan uzoqlashib ketgan.

2. `OneHotEncoder` - usuli yordamida har bir noyob qiymat alohida ustunga aylanadi va mos keluvchi ustunga 1, qolganlariga 0 qo'yiladi. Tushunarli bo'lishi uchun keyingi kodga ahamiyat bering.

In [ ]:
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot=cat_encoder.fit_transform(housing_cat)
housing_cat_1hot.toarray()

Yuqorida nima bo'ldi, bizda 5 ta kategoriya bor edi ('<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN').

OneHotEncoder 5 ta ustundan iborat yangi matrisa yaratdi. Har bir ustun yuqoridagi bitta kategoriyaga mos keladi: 1-ustun:'<1H OCEAN', 2-ustun:'INLAND' va hokazo.
Endi, har bir qatordagi matnning qiymatiga qarab mos keluvchi ustunga 1 qo'yiladi (qolgani 0 bo'ladi).

Masalan, `housing_cat` dataset 1-qatorini ko'rsak, `<1H OCEAN` qiymati turibdi, demak matrisaning 1-ustuni `1` qolganlari `0` bo'ladi. Buni yuoqridagi katakda ko'rishimiz mumkin.

In [ ]:
housing_cat.head(10)

Birinchi yoki ikkinchi usuldan qayerda foydalanish sizga havola. Agar matnli qiymatlarning turi judayam ko'p bo'lsa 2-usul datasetning kattalshib ketishiga sabab bo'ladi. Yuqorida, bizda 5 ta kategoriya bo'lgani uchun, datasetga yana 5 ta ustun qo'shishimiz kerak, bizda 16512 qator borligini hisobga olsak bu 82560 ta qo'shimcha qiymat degani.

Demak kategoriyalar soni oshgan sari, ishlov berish kerak bo'lgan ustunlar ham oshib ketadi bu esa ortiqcha resurs ishlatilishiga va qo'shimcha moliyaviy harajatlarga olib keladi.

### 3.3 Transfomer

Yuqorida ma'lumotlarg ishlov berishda biz `sklearn` tarkibidagi tayyor obyektlar va ulardagi `.fit()` yoki `.fit_transform()` funksiyalariga murojaat qildik.
Bu funksiyalar (`imputer`, `OneHotEncoder` va hokazo) **transformer**lar deb ataladi.

Umuman olganda `sklearn` tarkibidagi obyektlar uch turli bo'ladi:
1. **Estimator** - tahmin qiluvchi (baholovchi) funksiyalar. Bunday obyektlar `.fit()` metodiga ega bo'ladi, bu metod berilgan ma'lumotlardan qandaydur qiymatni hisoblaydi (masalan, avvalroq biz `imputer.fit()` yordamida median qiymatni hisobladik)
2. **Transformer** - ham tahmin qilib ham ma'lumotga ishlov beruvchi obyekt. Bunday obyektlar `.fit_transform()` metodiga ega bo'ladi, ya'ni avval qandaydur qiymatnbi hisoblaydi, keyin esa hisoblashni ma'lumotlarga qo'llaydi. Yuoqirdagi `OrdinalEncoder` va `OneHotEncoder` shunday obyektlardan.
3. **Predictor** - bashorat qiluvchi obyektlar. Bunday obyektlar `.predict()` metodiga ega bo'ladi. Ular biz bergan ma'lumotlar asosida bashoratlar qaytaradi. Dars davomida predictor obyektlar bilan ko'p ishlaymiz.

Obyektlarning afzalligi, ular hisob kitob natijalarini ichki attributrlarda saqlab qoladi. Masalan, yuqorida ko'rganimiz `imputer.statistics_`.

Keling endi o'zimiz ham transformer yozishni o'rganamiz.

Esingizda bo'lsa, avvalgi darsimizda biz ma'lumotlarg yangi 2 ustun qo'shdik: `rooms_per_household` va `bedrooms_per_room`.

Keling berilgan ma'lumotlarga shu ikki ustunni avtomat ravishda qo'shadigan transformer yasaymiz.

Buning uchun `sklearn` tarkibida `BaseEstimator` va `TransformerMixin` klasslaridan meros olib yangi klass yasaymiz va bu klassimizga kabi `fit()` va `transform()` metodlarini qo'shamiz:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

rooms_ix, bedrooms_ix, population_ix, household_ix=3,4,5,6

class ConditionalAttributesAdder(BaseEstimator, TransformerMixin):
  def __init__(self, add_bedroom_per_room=True):
    self.add_bedroom_per_room=add_bedroom_per_room

  def fit(self, X, y=None):
    return self
  def transform(self,X):
    rooms_per_houshold=X[:, rooms_ix]/X[:,household_ix]
    population_per_houshold=X[:,population_ix]/X[:,household_ix]
    if self.add_bedroom_per_room:
      bedroom_per_room= X[:, bedrooms_ix] / X[:, rooms_ix]
      return np.c_[X, rooms_per_houshold, population_per_houshold,bedroom_per_room]
    else:
      return np.c_[X, rooms_per_houshold, population_per_houshold]



In [ ]:
attr_adder=ConditionalAttributesAdder(add_bedroom_per_room=False)
housing_extra_attribs=attr_adder.transform(housing.values)
housing_extra_attribs[:,0]

In [ ]:
housing_extra_attribs

In [ ]:
attr_adder=ConditionalAttributesAdder(add_bedroom_per_room=True)
housing_extra_attribs=attr_adder.transform(housing.values)
housing_extra_attribs[:,0]

### 3.4 Standartizasiya va normalizasiya

Ahamiyat bersangiz, bizdagi ustunlardagi qiymatlar diapazoni turlicha, masalan:
- `housing_median_age` - `0-50` oralig'ida
- `total_rooms` esa `6-39000` oralig'ida
- `median_income` - `0.5-15` oralig'ida

Bularning hammasi kelgusi hisob kitoblarda algoritmlarning to'gri ishlashiga ta'sir ko'rsatadi.

Buning oldini olish uchun barcha ustundagi qiymatlarni bir diapazonga keltirib olish tavsiya qilinadi.

Buning uchun ML da min-max normallashtirish (_min-max scaling_) yoki standartlash (_standartization_) qo'llaniladi.

#### 3.4.1 Min-max scaling

Min-max scaling berilgan qiymatlarni 0-1 oralig'iga olib keladi.

Buning uchun biror ustundagi barcha qiymatlarning min ($x_{min}$) va max ($x_{max}$) qiymatlari topiladi. Keyin esa ustundagi har bir qiymat ($x$) quyidagi formula asosida hisoblab chiqiladi:

![](https://arshpreetsingh.files.wordpress.com/2017/03/normal.png)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max__scaler=MinMaxScaler()
min_max__scaler.fit_transform(housing_num)

#### 3.4.2 Standard Scaler - standartlash
StandardScaler biroz boshqa usulda ishlaydi. Sonlarni standartlas hcuhn bizga berilgan ustun bo'yicha o'rta qiymat ($\mu$) va standart cheklanish ($\sigma$) talab qilinadi. Har bir qatordag qiymat esa quydagicha hisoblanadi

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQxeMnm1BSa8qyNkCB6wqiYa790XtNxCdM48k_be-9zESUvivv_nuY6SXk8fpJK_psGiA&usqp=CAU)

In [ ]:
from sklearn.preprocessing import StandardScaler
standart_scaler = StandardScaler()
standart_scaler.fit_transform(housing_num)

In [ ]:
housing.head()

In [ ]:
housing_onehot=pd.get_dummies(housing['ocean_proximity'])
housing_onehot

In [ ]:
housing_num.head()